# LLM Part Dialogue — Bringing Parts to Life

In a real IFS session, the most powerful moment is when a client stops talking *about* a Part and starts talking *as* that Part. The Protector speaks in first person, reveals its fears, explains its strategies. The Exile whispers from the age when the wound was formed.

V2 of `agentic-ifs` makes this computational. The **LLM Part Dialogue** system takes each Part's narrative, age, intent, and strategies, and constructs a system prompt that instructs an LLM to speak *as* that Part. The Part's voice is grounded in its IFS data — not generic chat.

This notebook demonstrates:
1. Setting up the dialogue system with **Google Gemini** (via `GENAI_API_KEY`)
2. Speaking *as* a Part through the Self system (`speak_as`)
3. Direct Access mode — therapist speaks directly to a Part, bypassing Self
4. Multi-turn conversation with memory
5. The full therapeutic arc: dialogue during the 6 Fs and unburdening

## Setup

1. Copy `.env.example` to `.env.local` in the project root
2. Add your Google API key: `GENAI_API_KEY=your-key-here`
3. Run this notebook

> **Disclaimer:** `agentic-ifs` is a research and simulation tool. It is **not clinical software** and is not intended for therapy delivery.

In [ ]:
from pathlib import Path
from dotenv import load_dotenv

# Load .env.local from project root
load_dotenv(Path("../.env.local"))

import os
assert os.environ.get("GENAI_API_KEY"), (
    "GENAI_API_KEY not found. Copy .env.example to .env.local and add your key."
)
print("GENAI_API_KEY loaded.")

In [ ]:
from agentic_ifs import (
    Session,
    Manager,
    Firefighter,
    Exile,
    Burden,
    BurdenType,
    Edge,
    EdgeType,
    build_part_system_prompt,
)
from agentic_ifs.integrations.google import GeminiDialogueProvider

## 1. Build the Internal System

We create a realistic internal system: an **Inner Critic** (Manager) and a **Procrastinator** (Firefighter) both protecting a **Wounded Child** (Exile) who carries a burden of shame.

The `GeminiDialogueProvider` reads `GENAI_API_KEY` from the environment automatically.

In [ ]:
# Create the LLM provider — reads GENAI_API_KEY from environment
provider = GeminiDialogueProvider(model_name="gemini-2.5-flash")

# Create a Session with the dialogue provider
session = Session(
    initial_self_energy=0.8,
    dialogue_provider=provider,
)

# Build Parts
critic = Manager(
    narrative="The Inner Critic — formed at age 12 after being humiliated in front of the class",
    age=12,
    intent="Keep us safe from criticism by being perfect first",
    triggers=["criticism", "perceived failure", "being watched"],
    strategies=["perfectionism", "over-preparation", "self-criticism"],
    rigidity=0.8,
)

procrastinator = Firefighter(
    narrative="The Procrastinator — shuts everything down when the pressure gets too high",
    age=14,
    intent="Protect from overwhelm by stopping all effort",
    pain_threshold=0.6,
    extinguishing_behaviors=["avoidance", "distraction", "numbing with screens"],
)

wounded_child = Exile(
    narrative="The Wounded Child — carries shame from being humiliated at school",
    age=7,
    intent="Hold the pain so the system can function",
    emotional_charge=0.8,
    burden=Burden(
        burden_type=BurdenType.PERSONAL,
        origin="Age 7, humiliated in front of the class",
        content="I am not good enough",
    ),
)

# Add to session and wire relationships
session.add_part(critic)
session.add_part(procrastinator)
session.add_part(wounded_child)

session.add_edge(Edge(
    source_id=critic.id,
    target_id=wounded_child.id,
    edge_type=EdgeType.PROTECTS,
))
session.add_edge(Edge(
    source_id=procrastinator.id,
    target_id=wounded_child.id,
    edge_type=EdgeType.PROTECTS,
))

print(f"Internal system: {len(session.graph.nodes)} Parts, {len(session.graph.edges)} edges")
print(f"Self-energy: {session.self_system.self_energy}")
print(f"Self-led: {session.is_self_led}")

## 2. Inspect the System Prompt

Before we talk to a Part, let's see what the LLM actually receives. The system prompt is built from the Part's IFS data — narrative, age, intent, strategies, trust level. This is what grounds the LLM response in the Part's character.

In [ ]:
# System prompt for the Inner Critic
prompt = build_part_system_prompt(critic)
print("=== Inner Critic System Prompt ===")
print(prompt)
print()

# System prompt for the Wounded Child
prompt_exile = build_part_system_prompt(wounded_child)
print("=== Wounded Child System Prompt ===")
print(prompt_exile)

## 3. speak_as — Self Speaks to a Part

In IFS, the facilitator (Self) approaches a Part with curiosity and compassion. `speak_as()` checks that Self-energy is above the compassion threshold before engaging — if another Part has blended, the system cannot hold the space.

Let's ask the Inner Critic what its job is.

In [ ]:
# Self speaks to the Inner Critic
response = session.speak_as(
    critic.id,
    "I notice you're very active right now. Can you tell me about your job in this system?",
)

print("Inner Critic responds:")
print(response)

In [ ]:
# Follow-up: ask what it's afraid would happen if it stopped
response2 = session.speak_as(
    critic.id,
    "I appreciate what you do for us. What are you afraid would happen if you stopped criticising?",
)

print("Inner Critic responds:")
print(response2)

## 4. Multi-Turn Memory

Each Part maintains its own conversation history. Subsequent calls to `speak_as()` carry the full history, so the Part remembers what was said and responds in context — just like a real therapeutic conversation builds over time.

In [ ]:
# Third turn — the Part has memory of the previous two exchanges
response3 = session.speak_as(
    critic.id,
    "Who is it that you're trying to protect?",
)

print("Inner Critic responds:")
print(response3)

# Show the conversation history
print("\n--- Conversation history ---")
history = session.dialogue.get_history(critic.id)
for msg in history:
    label = "Self" if msg.role == "facilitator" else "Inner Critic"
    print(f"\n[{label}]: {msg.content}")

## 5. Direct Access — Therapist Bypasses Self

In IFS, **Direct Access** is a technique where a skilled therapist speaks directly to a Part, bypassing the client's Self. This is used when:
- Self-energy is too low for the client to engage
- A Protector is too blended to step back
- The therapist needs to negotiate directly with a protective Part

In `agentic-ifs`, `direct_access()` skips the Self-energy check and adds a special instruction to the system prompt indicating a therapist is speaking directly.

In [ ]:
# Direct Access to the Procrastinator
response_da = session.direct_access(
    procrastinator.id,
    "Hi there. I'd like to speak with you directly. "
    "I understand you step in when things get too much. "
    "Can you tell me what you experience right before you take over?",
)

print("Procrastinator (Direct Access):")
print(response_da)

In [ ]:
# Check the Direct Access system prompt to see the difference
prompt_da = build_part_system_prompt(procrastinator, is_direct_access=True)
print("=== Direct Access System Prompt ===")
print(prompt_da)

## 6. Speaking to the Exile

The most delicate work in IFS is speaking with Exiles. The Wounded Child carries the burden — the belief "I am not good enough" — and speaks from age 7. The system prompt captures this developmental age, emotional charge, and burden content.

In real therapy, this requires high Self-energy — you need to hold space with compassion. The library enforces this gate.

In [ ]:
# Self speaks to the Wounded Child
response_exile = session.speak_as(
    wounded_child.id,
    "Hello little one. I'm here with you now. Can you tell me what happened?",
)

print("Wounded Child responds:")
print(response_exile)

In [ ]:
# Follow-up with the Exile
response_exile2 = session.speak_as(
    wounded_child.id,
    "I'm so sorry that happened to you. What do you need from me right now?",
)

print("Wounded Child responds:")
print(response_exile2)

## 7. Session Log

Every dialogue exchange is logged in the `SelfSystem`'s session log. This provides a full audit trail of the therapeutic process — which Parts were engaged, what was said, and whether it was through Self or via Direct Access.

In [ ]:
# Review the session log
print(f"Session log entries: {len(session.self_system.session_log)}")
print()
for entry in session.self_system.session_log:
    print(f"[{entry.event_type}] {entry.timestamp.strftime('%H:%M:%S')}")
    # Truncate long descriptions for readability
    desc = entry.description
    if len(desc) > 120:
        desc = desc[:120] + "..."
    print(f"  {desc}")
    print()

## 8. Using a Different Provider

The dialogue system is **provider-agnostic**. The `DialogueProvider` protocol is a simple interface — any object with a `generate_part_response()` method works. You can use:

- `GeminiDialogueProvider` — Google Gemini (this notebook)
- `AnthropicDialogueProvider` — Anthropic Claude (`pip install agentic-ifs[anthropic]`)
- **Your own provider** — implement the protocol for any LLM, local model, or rule-based system

Here's a minimal mock provider to show the interface:

In [ ]:
from agentic_ifs import DialogueProvider, DialogueContext, PartDialogue
from agentic_ifs.parts import IPart


class EchoProvider:
    """A mock provider that echoes the Part's intent.
    
    Shows how simple the DialogueProvider interface is — just
    implement generate_part_response().
    """
    def generate_part_response(
        self, part: IPart, context: DialogueContext, system_prompt: str,
    ) -> str:
        return (
            f"[{part.part_type.title()} Part, age {part.age}] "
            f"My job is to {part.intent.lower()}. "
            f"You said: '{context.facilitator_message}'"
        )


# Use the echo provider
echo_session = Session(
    initial_self_energy=0.8,
    dialogue_provider=EchoProvider(),
)
echo_session.add_part(critic)

echo_response = echo_session.speak_as(critic.id, "How are you feeling?")
print(echo_response)

## Summary

The LLM Part Dialogue system bridges the computational IFS model and natural language:

| Feature | Method | Description |
|---|---|---|
| **Speak as Part** | `session.speak_as(part_id, message)` | Self speaks to a Part; requires Self-energy above compassion threshold |
| **Direct Access** | `session.direct_access(part_id, message)` | Therapist speaks directly to a Part, bypassing Self |
| **System prompts** | `build_part_system_prompt(part)` | IFS-grounded prompts from Part data (narrative, age, intent, strategies) |
| **Conversation memory** | `session.dialogue.get_history(part_id)` | Multi-turn dialogue with per-Part history |
| **Session logging** | `session.self_system.session_log` | Full audit trail of all dialogue exchanges |
| **Provider agnostic** | `DialogueProvider` protocol | Plug in any LLM backend — Gemini, Claude, local models, or your own |

The key insight: **the LLM is a rendering engine, not the IFS model.** The Part's character is encoded in its data fields. The LLM gives it a voice.

**Next:** Try [07_unburdening.ipynb](07_unburdening.ipynb) to see the full therapeutic arc — from Protector dialogue through the unburdening pipeline.